<a href="https://colab.research.google.com/github/vaibhaviitkgp/Assignment-3/blob/main/21BT10002_Asgn3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
import numpy as np
import pandas as pd

In [15]:
data = pd.read_csv("/content/housing.csv")
data.head()

,RM,LSTAT,PTRATIO,MEDV
0,6.575,4.98,15.3,504000.0
1,6.421,9.14,17.8,453600.0
2,7.185,4.03,17.8,728700.0
3,6.998,2.94,18.7,701400.0
4,7.147,5.33,18.7,760200.0


In [16]:
data_normalized = (data - data.min()) / (data.max() - data.min())
X = data_normalized.drop('MEDV', axis=1).values
y = data_normalized['MEDV'].values.reshape(-1, 1)
X.shape, y.shape

((489, 3), (489, 1))

In [17]:
def sigmoid(x):
    return 1.0 / (1.0 + np.exp(-x))

def sigmoid_derivative(x):
    return x * (1.0 - x)

class NeuralNetwork:
    def __init__(self, input_neurons, hidden_neurons, output_neurons, learning_rate):
        self.weights_input_hidden = np.random.rand(input_neurons, hidden_neurons)
        self.weights_hidden_output = np.random.rand(hidden_neurons, output_neurons)

        self.bias_hidden = np.random.rand(1, hidden_neurons)
        self.bias_output = np.random.rand(1, output_neurons)

        self.learning_rate = learning_rate

    def feedforward(self, X):
        self.hidden_layer_input = np.dot(X, self.weights_input_hidden) + self.bias_hidden
        self.hidden_layer_output = sigmoid(self.hidden_layer_input)

        self.output_layer_input = np.dot(self.hidden_layer_output, self.weights_hidden_output) + self.bias_output
        self.output = sigmoid(self.output_layer_input)
        return self.output

    def backpropagation(self, X, y):
        output_error = y - self.output
        output_delta = output_error * sigmoid_derivative(self.output)

        hidden_layer_error = output_delta.dot(self.weights_hidden_output.T)
        hidden_layer_delta = hidden_layer_error * sigmoid_derivative(self.hidden_layer_output)

        self.weights_hidden_output += self.hidden_layer_output.T.dot(output_delta) * self.learning_rate
        self.bias_output += np.sum(output_delta, axis=0, keepdims=True) * self.learning_rate

        self.weights_input_hidden += X.T.dot(hidden_layer_delta) * self.learning_rate
        self.bias_hidden += np.sum(hidden_layer_delta, axis=0, keepdims=True) * self.learning_rate

    def train(self, X, y, epochs):
        for epoch in range(epochs):
            self.feedforward(X)
            self.backpropagation(X, y)

    def predict(self, X):
        return self.feedforward(X)

In [18]:
def mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

configs = [
    {"hidden_neurons": 3, "learning_rate": 0.01},
    {"hidden_neurons": 4, "learning_rate": 0.001},
    {"hidden_neurons": 5, "learning_rate": 0.0001}
]

results = []

# Split the data into training and testing sets (80% train, 20% test)
train_size = int(0.8 * len(X))
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Train and evaluate the model for each configuration
for config in configs:
    nn = NeuralNetwork(input_neurons=3,
                       hidden_neurons=config["hidden_neurons"],
                       output_neurons=1,
                       learning_rate=config["learning_rate"])
    nn.train(X_train, y_train, epochs=1000)
    predictions = nn.predict(X_test)
    loss = mse(y_test, predictions)
    results.append((config, loss))

results


[({'hidden_neurons': 3, 'learning_rate': 0.01}, 0.008150075505167857),
 ({'hidden_neurons': 4, 'learning_rate': 0.001}, 0.03610471061625243),
 ({'hidden_neurons': 5, 'learning_rate': 0.0001}, 0.03497321243011294)]

In [21]:
def cross_validation(X, y, n_folds, hidden_neurons, learning_rate):
    fold_size = len(X) // n_folds
    losses = []

    for i in range(n_folds):
        # Split data into training and validation sets based on the current fold
        start, end = i * fold_size, (i + 1) * fold_size
        X_valid, y_valid = X[start:end], y[start:end]
        X_train = np.vstack((X[:start], X[end:]))
        y_train = np.vstack((y[:start], y[end:]))

        # Initialize and train the neural network
        nn = NeuralNetwork(input_neurons=3,
                           hidden_neurons=hidden_neurons,
                           output_neurons=1,
                           learning_rate=learning_rate)
        nn.train(X_train, y_train, epochs=1000)

        # Predict and calculate the loss for the validation set
        predictions = nn.predict(X_valid)
        loss = mse(y_valid, predictions)
        losses.append(loss)

    return np.mean(losses), np.std(losses)  # Return average and standard deviation of losses

# Perform 5-fold and 10-fold cross validation for the third configuration
five_fold_avg_loss, five_fold_std = cross_validation(X, y, n_folds=5, hidden_neurons=5, learning_rate=0.0001)
ten_fold_avg_loss, ten_fold_std = cross_validation(X, y, n_folds=10, hidden_neurons=5, learning_rate=0.0001)

five_fold_avg_loss, five_fold_std, ten_fold_avg_loss, ten_fold_std


(0.03722947303294825,
 0.0200555031874445,
 0.0355255641962495,
 0.022892700221199713)

5-fold,10-fold CV errors for 3rd configuration.

5-fold,10-fold CV errors for 2nd configuration.

In [24]:
five_fold_avg_loss, five_fold_std = cross_validation(X, y, n_folds=5, hidden_neurons=4, learning_rate=0.001)
ten_fold_avg_loss, ten_fold_std = cross_validation(X, y, n_folds=10, hidden_neurons=4, learning_rate=0.001)
five_fold_avg_loss, five_fold_std, ten_fold_avg_loss, ten_fold_std

(0.03459301698516315,
 0.01902417194729006,
 0.032692606874269545,
 0.021064276988637855)

5-fold,10-fold CV errors for 1st configuration.

In [23]:
five_fold_avg_loss, five_fold_std = cross_validation(X, y, n_folds=5, hidden_neurons=3, learning_rate=0.01)
ten_fold_avg_loss, ten_fold_std = cross_validation(X, y, n_folds=10, hidden_neurons=3, learning_rate=0.01)
five_fold_avg_loss, five_fold_std, ten_fold_avg_loss, ten_fold_std

(0.009063305939398695,
 0.0038966629840482907,
 0.008320862620738468,
 0.003870837353724008)

•	Using 5-fold cross-validation for 3rd configuration:

  •	Average MSE: ~0.0372
  
  •	Standard Deviation of MSE: ~0.0200

•	Using 10-fold cross-validation for 3rd configuration:

  •	Average MSE: ~0.0355

  •	Standard Deviation of MSE: ~0.0229


To summarize the results for all configurations:

(a) 3 neurons in the hidden layer, learning rate = 0.01: MSE = ~0.00815

(b) 4 neurons in the hidden layer, learning rate = 0.001: MSE = ~0.0361

(c) 5 neurons in the hidden layer, learning rate = 0.0001: MSE = ~0.0350


These results suggest that the first configuration performs the best out of the three.